In [1]:
import pandas as pd 
import matplotlib as plto
import re as re 
import random as rand
import numpy as np
import time 

pd.set_option('display.max_colwidth', -1)
pd.set_option('mode.chained_assignment', 'warn')

col_names = ['chromosome', 'source', 'type', 'start', 'end', 'score', 'strand', 'phase', 'attributes'] # For df object
#df = data frame (object of the Homo Sapians file)
df = pd.read_csv('Homo_sapiens.GRCh38.85.gff3.gz', compression='gzip',
                         sep='\t', comment='#', low_memory=False,
                         header=None, names=col_names)

In [ ]:
# Made this function in a better way below 
# def closestGeneBody(chrNum, site, window):
#     out = ['No Result', 9**90] # This is just a random num that will be replace by func below. 
#     geneDF = geneAnnotations.loc[geneAnnotations['chromosome'] == str(chrNum)]
    
#     for index, row in geneDF.iterrows():
#         if inRange(row['start'], site, window):
#             if isNearBody(row['start'], row['end'], site, window):
#                 if distanceBody(row['start'], row['end'], site) < out[1]:
#                     out[0] = row['gene_name']
#                     out[1] = distanceBody(row['start'], row['end'], site)
#         else: 
#             return(out)
#     # Returns closest gene and its distance from body. 
#     return(out)

# finds unique in list | Might use later 
# def unique(list1): 
#     unique_list = [] 
#     for x in list1: 
#         if x not in unique_list: 
#             unique_list.append(x) 
#     return unique_list

# A. Estimating number of sites to be sampled randomly

# <font color=#4286f4>Setting up my DataFrame as variable: genes/chromosome</font>

In [2]:
geneAnnotations = df[df.type.isin(['gene'])].copy()
chromosome = df[df.type.isin(['chromosome'])].copy()

# <font color=#4286f4>Adding Columns to geneAnnotations: gene_name, gene_id, length</font>

In [3]:
# ONLY NEEDS TO BE DONE ONCE!!! (BOTH UP AND DOWN)
RE_GENE_NAME = re.compile(r'Name=(?P<gene_name>.+?);') 
#|Below| Searching for data in attributes col and then creating new col for that data
def extract_gene_name(attributes_str): 
    res = RE_GENE_NAME.search(attributes_str)
    return res.group('gene_name')
geneAnnotations['gene_name'] = geneAnnotations.attributes.apply(extract_gene_name)

In [4]:
RE_GENE_ID = re.compile(r'gene_id=(?P<gene_id>ENSG.+?);')
def extract_gene_id(attributes_str): # New COL again
    res = RE_GENE_ID.search(attributes_str)
    return res.group('gene_id')
geneAnnotations['gene_id'] = geneAnnotations.attributes.apply(extract_gene_id)

In [5]:
geneAnnotations['length'] = geneAnnotations.end - geneAnnotations.start + 1

In [6]:
geneAnnotationsDF = geneAnnotations.drop(['attributes'], axis=1)
chromosomesDF = chromosome.drop(['attributes'], axis=1)

# <font color=#4286f4>Creating DataFrame for n Chromosomes with n sites </font>

In [7]:
#I need random chromosome combined with random position (Somewhere between start and end)
def nBindingsites(n):    
    randChr = chromosome.sample(n, replace=True)
    randChr['BindingSite'] = (np.random.uniform(1, randChr.end).round()).astype(int)
    return randChr
    #Returns a DataFrame with Random sample number with random BindingSite

In [8]:
#MIGHT NOT NEED ANY OF THIS ACTUALLY COULD BE FASTER 

# Distance from Body
def distanceBody(start, end, site):
    return min(abs(start - site), abs(end - site))

# Distance from TSS
def distanceTSS(start, site):
    return abs(start - site)

#Returns True or False to wheather the binding site is within the window size to Body | Start Site below
def isNearBody(start, end, site, window):
    if abs(start - site) <= window:
        return True
    elif abs(end - site) <= window:
        return True
    else:
        return False 
    
def isNearTSS(start, site, window):
    if abs(start - site) <= window:
        return True
    else: 
        return False
#-------------------------------------------- ## Should there be anymore iterations? Checker function 
def inRange(start, site, window):
    if start > (site + window):
        return False 
    else:
        return True

# For: k = Infinity 

In [9]:
#Given specific chromosome, rdm binding site, window: Will calculate number of nearby gene(bodies) from its site.
def genesInWindowBody(chrNum, site, window):
    count = 0
    geneL = []
    geneDF = geneAnnotations.loc[geneAnnotations['chromosome'] == str(chrNum)]
    
    for index, row in geneDF.iterrows():
        if isNearBody(row['start'], row['end'], site, window):
            count = count + 1
            geneL.append(row['gene_name'])
    return(count, geneL)

#Given specific chromosome, rdm binding site, window: Will calculate number of nearby gene(TSS) from the site.
def genesInWindowTSS(chrNum, site, window):
    count = 0
    geneL = []
    geneDF = geneAnnotations.loc[geneAnnotations['chromosome'] == str(chrNum)]
    
    for index, row in geneDF.iterrows():
        if isNearTSS(row['start'], site, window):
            count = count + 1
            geneL.append(row['gene_name'])    
    return(count, geneL)

## <font color=#4286f4>Given a window: Find all the genes inside window for the dataframe of chromosomes</font>

In [10]:
# Takes DF and Window size: Returnes Total Gene count and List of Genes Near Body of GENE (K=INFINITY)
def geneCountNsitesBody(df, window):
    count = 0 
    geneL = []
    
    for index, row in df.iterrows():
        c = genesInWindowBody(row['chromosome'], row['BindingSite'], window)
        count = count + c[0]
        geneL.append(c[1])
    return(count, geneL)

# Takes DF and Window size: Returnes Total Gene count and List of Genes Near TSS of GENE (K=INFINITY)
def geneCountNsitesTSS(df, window):
    count = 0 
    geneL = []
    
    for index, row in df.iterrows():
        c = genesInWindowTSS(row['chromosome'], row['BindingSite'], window)
        count = count + c[0]
        geneL.append(c[1])
    return(count, geneL)

# For: k = 1

In [52]:
#Given specific chromosome, rdm binding site, window: Will calculate closest gene(body) from its site. and distance
def closestGeneBody(chrNum, site, window):
    out = ['No Result', 9**90] # This is just a random num that will be replace by func below. 
    geneDF = geneAnnotations.loc[geneAnnotations['chromosome'] == str(chrNum)]
    windowDF = geneDF[(abs(geneDF.start - site) < window) | (abs(geneDF.end - site) < window)]
    
    for index, row in windowDF.iterrows():
        if distanceBody(row['start'], row['end'], site) < out[1]:
            out[0] = row['gene_name']
            out[1] = distanceBody(row['start'], row['end'], site)

    # Returns closest gene and its distance from body. 
    return(out)

#Given specific chromosome, rdm binding site, window: Will calculate closest gene(TSS) from its site. and Distance
def closestGeneTSS(chrNum, site, window):
    out = ['No Result', 9**90]
    geneDF = geneAnnotations.loc[geneAnnotations['chromosome'] == str(chrNum)]
    
    for index, row in geneDF.iterrows():
        if isNearTSS(row['start'], site, window):
            if distanceTSS(row['start'], site) < out[1]:
                out[0] = row['gene_name']
                out[1] = distanceTSS(row['start'], site)
    # Returns closest gene and its distance from TSS. 
    return(out)

In [50]:
# Takes DF and Window size: Returnes Total Gene count and List of CLOSEST Gene Near its BODY for every site(K = 1)
def closestGenesNsitesBody(df, window):
    count = 0 
    geneL = []
    
    for index, row in df.iterrows():
        x = closestGeneBody(row['chromosome'], row['BindingSite'], window)
        geneL.append(x[0])
        count = count + 1
    return(count, geneL)
    
# Takes DF and Window size: Returnes Total Gene count and List of CLOSEST Gene Near its TSS for every site (K = 1)
def closestGenesNsitesTSS(df, window):
    count = 0 
    geneL = []
    
    for index, row in df.iterrows():
        x = closestGeneTSS(row['chromosome'], row['BindingSite'], window)
        geneL.append(x[0])
        count = count + 1
    return(count, geneL)

In [36]:
geneOne = geneAnnotationsDF[geneAnnotationsDF.chromosome == '2']
geneOne[(abs(geneOne.start - 37000) < 2000) | (abs(geneOne.end - 200000) < 3000)]

,chromosome,source,type,start,end,score,strand,phase,gene_name,gene_id,length
1364670,2,ensembl_havana,gene,38814,46870,.,-,.,FAM110C,ENSG00000184731,8057
1364709,2,havana,gene,197569,202605,.,+,.,AC079779.7,ENSG00000227061,5037


In [27]:
geneAnnotationsDF[geneAnnotationsDF.chromosome == '2'][geneAnnotationsDF.start < 200900]

/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  """Entry point for launching an IPython kernel.


,chromosome,source,type,start,end,score,strand,phase,gene_name,gene_id,length
1364670,2,ensembl_havana,gene,38814,46870,.,-,.,FAM110C,ENSG00000184731,8057
1364709,2,havana,gene,197569,202605,.,+,.,AC079779.7,ENSG00000227061,5037


In [35]:
# 145520 < x < 263618
202605-200000

2605